In [1]:
import xml.dom.minidom as dom
import pandas as pd
import numpy as np
import sys
from math import cos, asin, sqrt
from pykalman import KalmanFilter
import difflib
import matplotlib.pyplot as plt
import folium
from scipy import stats
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 1000)
# pd.set_option('display.width', 10)
# REF for maps function: https://www.zhihu.com/question/33783546

In [3]:
# df = pd.read_json(sys.argv[1])
df = pd.read_json('amenities-vancouver.json.gz', lines=True)
df = df.dropna()
# print(df)

restaurant = df[df['amenity'].str.contains('restaurant', case=False)]
restaurant_count = restaurant['name'].value_counts() 
restaurant_chain = restaurant_count[restaurant_count>=2]
restaurant_chain = pd.DataFrame({'name': restaurant_chain.index, 'count': restaurant_chain.values})

fastfood = df[df['amenity'].str.contains('food', case=False)]
fastfood_count =  fastfood['name'].value_counts()
fastfood_chain = fastfood_count[fastfood_count>=2]

restaurant_chain_all = df[df['amenity'].str.contains('food|restaurant', case=False)]
# print(restaurant_chain_all)

fastfood_chain = pd.DataFrame({'name': fastfood_chain.index, 'count': fastfood_chain.values})
result_chain = restaurant_chain.append(fastfood_chain).sort_values(by='count', ascending=False).reset_index(drop=True)
result_chain
# restaurant_chain_all


,name,count
0,Subway,154
1,A&W,55
2,McDonald's,54
3,Tim Hortons,27
4,White Spot,27
5,Quiznos,23
6,KFC,22
7,Dairy Queen,20
8,Boston Pizza,18
9,Burger King,18


In [ ]:
restaurant_ind = restaurant_count[restaurant_count ==1]
restaurant_ind = pd.DataFrame({'name': restaurant_ind.index, 'count': restaurant_ind.values})
fastfood_ind = fastfood_count[fastfood_count == 1]
fastfood_ind = pd.DataFrame({'name': fastfood_ind.index, 'count': fastfood_ind.values})
result_ind = restaurant_ind.append(fastfood_ind).reset_index(drop=True)
# result_ind

In [ ]:
chain_name = input("Please enter the restaurant name to search: ")
chain = df[df['name'].str.contains(chain_name, case=False)]
# print(mac)
chain_lab = chain['name']
chain_lat = chain['lat']
chain_lon = chain['lon']

all_chain_lab = restaurant_chain_all['name']
all_chain_lat = restaurant_chain_all['lat']
all_chain_lon = restaurant_chain_all['lon']

In [ ]:
# world_map = folium.Map()
# world_map
van_latitude =  49.246292
van_longitude = -123.116226
van_map = folium.Map(location=[van_latitude, van_longitude], zoom_start=12)
new_van_map = folium.Map(location=[van_latitude, van_longitude], zoom_start=12)

latitudes = list(chain_lat)
longitudes = list(chain_lon)
labels = list(chain_lab)

incidents = folium.map.FeatureGroup()

In [ ]:
latitudes_all_chain = list(all_chain_lat)
longitudes_all_chain = list(all_chain_lon)
labels_all_chain = list(all_chain_lab)

In [ ]:
for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(van_map)    
    
# add incidents to map
van_map.add_child(incidents)

In [ ]:
from folium.plugins import HeatMap

van_heat_map = folium.Map(location=[van_latitude, van_longitude], zoom_start=12)

van_density_map = folium.Map(location = [van_latitude, van_longitude], zoom_start = 12)

# Convert data format
heatdata = chain[['lat','lon']].values.tolist()

# add incidents to map
HeatMap(heatdata).add_to(van_heat_map)
# hmap.add_child(HeatMap(data, radius = 25, gradient={.4: 'blue', .65: 'lime', 1: 'red'}))

van_heat_map

In [ ]:
# for lat, lng, in zip(chain_lat, chain_lon):
#     incidents.add_child(
#         folium.CircleMarker(
#             [lat, lng],
#             radius=7, 
#             color='yellow',
#             fill=True,
#             fill_color='red',
#             fill_opacity=0.4
#         )
#     )
# van_map = folium.Map(location=[van_latitude, van_longitude], zoom_start=12)
# van_map.add_child(incidents)

In [ ]:
# # all chian marked
# for lat, lng, label in zip(latitudes_all_chain, longitudes_all_chain, labels_all_chain):
#     folium.Marker([lat, lng], popup=label).add_to(new_van_map)    
    
# # add incidents to map
# new_van_map.add_child(incidents)


In [ ]:

van_heat_map_all = folium.Map(location=[van_latitude, van_longitude], zoom_start=12)

van_density_map_all = folium.Map(location = [van_latitude, van_longitude], zoom_start = 12)

# Convert data format
heatdata_all = restaurant_chain_all[['lat','lon']].values.tolist()

# add incidents to map
HeatMap(heatdata_all).add_to(van_heat_map_all)

van_heat_map_all
